**Refffered from**
https://artificialcorner.com/gpt4all-is-the-local-chatgpt-for-your-documents-and-it-is-free-df1016bc335

**Mounting google drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Listing folders in mydrive**

In [2]:
!ls "/content/drive/MyDrive/"

'Colab Notebooks'   Docs_Chat   Sources


**Placing folder(Docs_Chat) with Sources(.txt,.pdf files) into new folder created in /content directory on collab**

In [3]:
import shutil

# Define the source and destination paths
source_folder = "/content/drive/MyDrive/Docs_Chat"
destination_folder = "/content/Docs_Chat"

# Copy the contents of the source folder to the destination folder
shutil.copytree(source_folder, destination_folder)

'/content/Docs_Chat'

**Going inside Docs_Chat folder**

In [4]:
%cd Docs_Chat

/content/Docs_Chat


**Setting up virtual environment**

In [5]:
!sudo apt install python3-setuptools
!sudo apt install python3.8-venv
!python3.8 -m venv .venv

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  python-setuptools-doc
The following NEW packages will be installed:
  python3-setuptools
0 upgraded, 1 newly installed, 0 to remove and 15 not upgraded.
Need to get 330 kB of archives.
After this operation, 1,467 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 python3-setuptools all 45.2.0-1ubuntu0.1 [330 kB]
Fetched 330 kB in 2s (213 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected 

**Activating Virtual Evnvironment**

In [6]:
!source .venv/bin/activate

**Installing required libraries**

In [7]:
!pip install sentence-transformers
!pip install langchain
!pip install pypdf
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=be64523ee688b4518fcbc9a443a5c175d4fa1c029699436ece94c7cfb7089d27
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.8 MB/s eta 0:

**Import necessary modules:**
The code begins by importing the required modules for document loading, embeddings, text splitting, and search indexing.

In [8]:
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores.faiss import FAISS
from langchain.document_loaders import DirectoryLoader
import os

**Create the HuggingFace embeddings object:**
An instance of the HuggingFaceEmbeddings class is created using the 'sentence-transformers/all-MiniLM-L6-v2' model. This will be used for embedding text.

In [9]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

**Define the split_chunks function:**
This function takes a list of documents and splits them into chunks using the RecursiveCharacterTextSplitter. The chunks are returned as a list.

In [10]:
# Split text into chunks
def split_chunks(sources):
    chunks = []
    splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=32)
    for chunk in splitter.split_documents(sources):
        chunks.append(chunk)
    return chunks

**Define the create_index function:**
This function takes a list of document chunks, extracts the page content and metadata, and creates a search index using the FAISS vector store. The created index is returned.

In [11]:
# Create the search index
def create_index(chunks):
    texts = [doc.page_content for doc in chunks]
    metadatas = [doc.metadata for doc in chunks]

    search_index = FAISS.from_texts(texts, embeddings, metadatas=metadatas)

    return search_index


**Define the similarity_search function:**
This function takes a query and a search index. It performs a similarity search using the index and retrieves the matched documents. The matched documents and their sources are returned.

In [12]:
# Search for similarity
def similarity_search(query, index):
    matched_docs = index.similarity_search(query, k=3)
    sources = []
    for doc in matched_docs:
        sources.append(
            {
                "page_content": doc.page_content,
                "metadata": doc.metadata,
            }
        )

    return matched_docs, sources


**Set the folder path and load documents:**
The folder_path variable is set to the folder containing the PDF and TXT files. The code then retrieves a list of the PDF and TXT files in the folder.

In [15]:
# Set the folder path for PDF and TXT files
folder_path = '/content/Docs_Chat/Sources'

**Load and process the documents:**
The code iterates over the list of files and loads each document using the appropriate loader (PyPDFLoader for PDF files and TextLoader for TXT files). The loaded documents are split into chunks using the split_chunks function and added to the docs list.

In [16]:
# Get the list of PDF and TXT files in the folder
doc_list = [s for s in os.listdir(folder_path) if s.endswith(('.pdf', '.txt'))]
num_of_docs = len(doc_list)

docs = []

for doc_file in doc_list:
    doc_path = os.path.join(folder_path, doc_file)
    if doc_file.endswith('.pdf'):
        loader = PyPDFLoader(doc_path)
        loaded_docs = loader.load()
        docs.extend(loaded_docs)
    elif doc_file.endswith('.txt'):
        loader = TextLoader(doc_path)
        loaded_docs = loader.load()
        docs.extend(loaded_docs)

chunks = split_chunks(docs)


**Create and merge search indexes:**
The create_index function is used to create a search index for the initial set of loaded documents (db0). Then, for each subsequent document, a new search index (dbi) is created and merged into db0 using the merge_from method. This combines all the search indexes into a single index.

In [17]:
db0 = create_index(chunks)
for i in range(1, num_of_docs):
    doc_file = doc_list[i]
    doc_path = os.path.join(folder_path, doc_file)

    if doc_file.endswith('.pdf'):
        loader = PyPDFLoader(doc_path)
        loaded_docs = loader.load()
    elif doc_file.endswith('.txt'):
        loader = TextLoader(doc_path)
        loaded_docs = loader.load()
    else:
        # Handle unsupported file types or unknown extensions
        print(f"Unsupported file type: {doc_file}")
        continue

    chunks = split_chunks(loaded_docs)
    dbi = create_index(chunks)

    db0.merge_from(dbi)



**Save the search index:**
The db0 search index is saved locally using the save_local method with the name "my_faiss_index".

In [18]:
# Save the database locally
db0.save_local("my_faiss_index")

**Load the search index:**
The saved search index is loaded from the local file using the load_local method, and the embeddings object is passed as an argument.

In [19]:
# Load our local index vector db
index = FAISS.load_local("my_faiss_index", embeddings)

**Perform similarity search:**
The code prompts the user to enter a question and then performs a similarity search on the loaded search index using the user's query. The results are stored in the docs variable.

In [20]:
query = input("Enter your question: ")
docs = index.similarity_search(query)

Enter your question: sumanvitha


**Print the results:**
The question and the best three matching results are printed to the console.

In [21]:
# Get the best match
print(f"The question is: {query}")
print(docs[0])

The question is: sumanvitha
page_content='There is another girl named Sumanvitha.\nCurrently doing intern at Microsoft.\nInter Period: 2months.\nIntern Stipend: 1,25,000.\nSumanvitha is currently pursuing btech in computer science department at MANIT bhpal.' metadata={'source': '/content/Docs_Chat/Sources/Sumanvitha.txt'}
